In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [6]:
import addict
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from okada_wrapper import dc3dwrapper

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [ ]:
import celeri_closure
celeri_closure = reload(celeri_closure)
import celeri
celeri = reload(celeri)

In [ ]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

In [9]:
success, u, grad_u = dc3dwrapper(0.6, [1.0, 1.0, -1.0], 3.0, 90, [-0.7, 0.7], [-0.7, 0.7], [1.0, 0.0, 0.0]) 